<a href="https://colab.research.google.com/github/gauthiermartin/pytorch-deep-learning-course/blob/main/08_pytorch_paper_replicating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08. PyTorch Paper Replicating

Turn a ML research paper into usable code

In this notebook we are going to be replicating the Vision Transformer (ViT)

Paper - https://arxiv.org/abs/2010.11929v2



# Anatomy of a research paper

1. Abstract : An *overview/summary* of the paper
2. Introduction: What is the paper's *main problem* ? And details of previous methods used to try and solve it.
3. Method: What *steps did the researchers take* to solve the problem? (Example: Model, Datasource, training setup)
4. Results: What are the *outcomes* of the paper? If a new type of model training setup was used how did the results compare to previous methods?
5. Conclusion: What are the *limitations* of the suggested methods ? What are the *future work* ?
6. References: What *resources/other papers* did the researchers use to build there own body of work?
7. Appendix: Are there any *extra resources/findings* that were not mentioned in the above section of the paper?


## Why replicate machine learning papers

If you want the skills to be a Machine Learning Engineer here are the steps

1. Download a paper
2. Implement it
3. Keep doing this until you have skills
4. Build infrastructure around your models / data preprocessing steps

## Where can we find machine learning research paper

- [arXiv](https://arxiv.org/)
- [AK](https://twitter.com/_akhaliq)
- [Paper With Code](https://paperswithcode.com/)
- [ViT PyTorch](https://github.com/lucidrains/vit-pytorch)

## What are we going to cover

* Getting setup (importing previously written code)
* Introduce a machien elarning paper replicating with PyTorch
* Replicating ViT for FoodVisionMini
* Training a custom ViWaitingForTextObjectMode
* Feature Extraction with a pretrained ViT




# 0. Get setup

In [1]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4033, done.
remote: Counting objects: 100% (1224/1224), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 4033 (delta 1067), reused 1100 (delta 999), pack-reused 2809
Receiving objects: 100% (4033/4033), 649.60 MiB | 39.23 MiB/s, done.
Resolving deltas: 100% (2358/2358), done.
Updating files: 100% (248/248), done.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# 1. Get Data

The whole goal of what we are trying to do here is to replicate ViT for our FoodVision Mini problem

In [3]:
# Download pizza, steak, sushi images from GitHub
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] Did not find data/pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


PosixPath('data/pizza_steak_sushi')

In [4]:
# Setup directory paths to train and test images
train_dir = image_path / "train"
test_dir = image_path / "test"